In [44]:
#| default_exp encoder

In [45]:
#| hide
#%load_ext autoreload --> Not working TODO:REVISAR
# %autoreload 2 

In [46]:
#| export
from dvats.memory import *
from dvats.utils import Time
from dvats.utils import print_flush

In [47]:
## -- Classes & types
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Callable

# Encoder

> Architectures and functions for creating encoders that create the embeddings

In [48]:
#| export
import pandas as pd
import numpy as np
from fastcore.all import *
from tsai.callback.MVP import *
from tsai.imports import *
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.models.explainability import get_acts_and_grads
from tsai.models.layers import *
from tsai.data.validation import combine_split_data
from fastai.callback.hook import hook_outputs
from momentfm import MOMENTPipeline
from gluonts.dataset.pandas import PandasDataset
import time
import einops

In [49]:
#| hide
from tsai.all import *

### Architectures

In [50]:
#|export 
class DCAE_torch(Module):
    def __init__(self, c_in, seq_len, delta, nfs=[64, 32, 12], kss=[10, 5, 5],
                 pool_szs=[2,2,3], output_fsz=10):
        """
        Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions,
        sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be
        contained in the Dense layer of the network. The the number of features
        maps (filters), the filter size and the pool size can also be adjusted."
        """
        assert all_equal([len(x) for x in [nfs, kss, pool_szs]], np.repeat(len(nfs), 3)), \
            'nfs, kss, and pool_szs must have the same length'
        assert np.prod(pool_szs) == nfs[-1], \
            'The number of filters in the last conv layer must be equal to the product of pool sizes'
        assert seq_len % np.prod(pool_szs) == 0, \
            'The product of pool sizes must be a divisor of the window size'
        layers = []
        for i in range_of(kss):
            layers += [Conv1d(ni=nfs[i-1] if i>0 else c_in, nf=nfs[i], ks=kss[i]),
                       nn.MaxPool1d(kernel_size=pool_szs[i])]
        self.downsample = nn.Sequential(*layers)
        self.bottleneck = nn.Sequential(OrderedDict([
            ('flatten', nn.Flatten()),
            ('latent_in', nn.Linear(seq_len, delta)),
            ('latent_out', nn.Linear(delta, seq_len)),
            ('reshape', Reshape(nfs[-1], seq_len // np.prod(pool_szs)))
        ]))
        layers = []
        for i in reversed(range_of(kss)):
            layers += [Conv1d(ni=nfs[i+1] if i != (len(nfs)-1) else nfs[-1],
                              nf=nfs[i], ks=kss[i]),
                       nn.Upsample(scale_factor=pool_szs[i])]
        layers += [Conv1d(ni=nfs[0], nf=c_in, kernel_size=output_fsz)]
        self.upsample = nn.Sequential(*layers)


    def forward(self, x):
        x = self.downsample(x)
        x = self.bottleneck(x)
        x = self.upsample(x)
        return x

In [51]:
#| hide
foo = torch.rand(3, 1, 48)
m = DCAE_torch(c_in=foo.shape[1], seq_len=foo.shape[2], delta=12)
m(foo).shape

torch.Size([3, 1, 48])

### Dictionary to get the default backbone modules to get the embeddings from

In [52]:
#| export
ENCODER_EMBS_MODULE_NAME = {
    InceptionTimePlus: 'backbone', # for mvp based models
    DCAE_torch: 'bottleneck.latent_in'#,
    #MoiraiForecast: 'mask_encoding' #TODO: check
    
}

### Get activations

In [53]:
#| export 
def kwargs_to_gpu_(**kwargs):
    for key in kwargs:
        try: #if not able to be moved, just not move it
            kwargs[key] = kwargs[key].to("cuda")
        except:
            continue
    
def kwargs_to_cpu_(**kwargs):
    for key in kwargs:
        try: #if not able to be moved, just not move it
            kwargs[key] = kwargs[key].cpu()
        except:
            continue
   

In [54]:
#| export
def get_acts(
    model : torch.nn.Module, 
    module: torch.nn.Module, 
    cpu   : bool, 
    verbose : int = 0,
    retry: bool = False,
    acts_indices: List [ int ] = None,
    **model_kwargs #Parameters of the model
):
    if verbose > 0:
        print_flush(f"--> get acts | acts indices: {acts_indices}")
    if cpu:
        if verbose > 0: print_flush(f"get acts | Moving to cpu")
        for key in model_kwargs:
            try: #if not able to be moved, just not move it
                model_kwargs[key] = model_kwargs[key].cpu()
            except:
                continue
        model.to("cpu")
    else:
        if verbose > 0: print_flush(f"get acts | Moving to gpu")
        for key in model_kwargs:
            try: #if not able to be moved, just not move it
                model_kwargs[key] = model_kwargs[key].to("cuda")
            except:
                continue
        model.to("cuda")
    if verbose > 0: print_flush(f"get acts | Add hooks")
    h_act = hook_outputs([module], detach = True, cpu = cpu, grad = False)
    with torch.no_grad():
        if verbose > 0: print_flush(f"get acts | --> Run forward")
        if retry:
            if verbose > 0: print_flush(f"get acts | Retry")
            try: 
                preds = model.eval()(**model_kwargs)
            except Exception as e:
                print_flush(f"get acts | Retry | Error: {e}")
                print_flush(f"get acts | Retry | Kwargs: {model_kwargs}")
                if not cpu:
                    print_flush(f"get acts | Retry | Moving to cpu")
                    for key in model_kwargs:
                        try: #if not able to be moved, just not move it
                            model_kwargs[key] = model_kwargs[key].cpu()
                        except:
                            continue
                    model.to("cpu")
                    if verbose > 0: print_flush(f"get acts | Retry | cpu")
                    print_flush(f"get acts | Retry | Get acts")
                    preds = model.eval()(**model_kwargs)
        else:
            if verbose > 2: print_flush(f"get acts | No Retry | Get acts | model kwargs: {model_kwargs}")
            preds = model.eval()(**model_kwargs)
    if acts_indices is None:
        res = [o.stored for o in h_act]
    else: 
        stored = [o.stored for o in h_act]
        res = [stored[i] for i in acts_indices]
        if len(acts_indices) == 1:
            res = res[0]
        del stored
    if verbose > 0: print_flush(f"get acts | Run forward -->")
    if verbose > 0:print_flush(f"get acts -->")
    return res

In [55]:
#| export
def get_acts_moment(enc_learn, cpu, verbose, y, mask = None, padd_step = 100, retry = False, max_trials = 5, acts_indices = [0]):
    success = False 
    trial = 0
    embs = None
    while not success and trial < max_trials:
        trial += 1
        try:
            if verbose > 0: print_flush(f"get_acts_moment | Trial {trial} | x_enc ~ {y.shape}")
            embs = get_acts(
                model = enc_learn,
                #module = enc_learn.encoder.dropout,
                module = enc_learn.head.dropout,
                cpu = cpu,
                verbose = 0,
                x_enc = y,
                retry = retry,
                acts_indices = acts_indices,
                mask = mask
            )
            success = True
            if verbose > 0 and acts_indices == [0] : print_flush(f"get_acts_moment | Trial {trial} | embs ~ {embs.shape}")
        except Exception as e:
            if trial == max_trials - 1 : raise
            if verbose > 0:
                print_flush(f"get_acts_moment | Trial {trial} | About to pad X (encoder input) | exception {e} | padd step: {padd_step}")
                print_flush(f"get_acts_moment | Trial {trial} | y ~ {y.shape}")
            if "tensor a" in str(e) and "tensor b" in str(e):
                match = re.search(r'tensor a \((\d+)\) must match the size of tensor b \((\d+)\)', str(e))
                tensor_a_size = int(match.group(1))
                tensor_b_size = int(match.group(2))
                padd = True
                if trial > 1: 
                    if verbose > 0: print_flush(f"------------------- Trial {trial}  -----------------")
                    if tensor_a_size > tensor_a_size_old:
                        if verbose > 0:  print_flush(f"------------------- Trial {trial} | a > a_old -----------------")
                        padd = False
                        y = y [ ..., : tensor_a_size - tensor_b_size]
                        if verbose > 0: print_flush(f"------------------- Trial {trial} |a > a_old | Reduced |  y ~ {y.shape} -----------------")
                if padd:
                    if verbose > 0: print_flush(f"------------------- Trial {trial} | Padd -----------------")
                    if tensor_a_size > tensor_b_size: 
                        if verbose > 0: print_flush(f"------------------- Trial {trial} | Padd | a > b -----------------")
                        padd_step = tensor_a_size - tensor_b_size
                    y = torch.nn.functional.pad(y,(0,padd_step))
                tensor_a_size_old = tensor_a_size
            else:
                if verbose > 0: print_flush("Not the usual error. No padding, just fail")
                raise
                
    return embs

In [56]:
#| export
def sure_eval_moment(enc_learn, cpu, verbose, y, input_mask = None, mask = None, padd_step = 100, retry = False, max_trials = 5, acts_indices = [0]):
    y.to("cpu")
    y_copy = y.clone()
    
    if verbose > 0: print_flush(f"---> sure_eval_moment")
    success = False 
    trial = 0
    output = None
    if cpu:
        enc_learn.to("cpu")
        y.to("cpu")
        if input_mask is not None: input_mask.to("cpu")
        if mask is not None: mask.to("cpu")
    else:
        enc_learn.to("cuda")
        y.to("cuda")
        if input_mask is not None: input_mask.to("cuda")
        if mask is not None: mask.to("cuda")
    while not success and trial < max_trials:
        trial += 1
        try:
            if verbose > 0: print_flush(f"sure_eval_moment | Trial {trial} | x_enc ~ {y.shape}")
            
            output = enc_learn(x_enc = y, input_mask = input_mask, mask = mask)
            success = True
            if verbose > 0 and acts_indices == [0] : print_flush(f"sure_eval_moment | Trial {trial} | embs ~ {embs.shape}")
        except Exception as e:
            if verbose > 0:
                print_flush(f"sure_eval_moment | Trial {trial} | About to pad X (encoder input) | exception {e} | padd step: {padd_step}")
                print_flush(f"sure_eval_moment | Trial {trial} | y ~ {y.shape}")
            if "tensor a" in str(e) and "tensor b" in str(e) and "dimension" in str(e):
                match = re.search(r'tensor a \((\d+)\) must match the size of tensor b \((\d+)\) at non-singleton dimension (\d+)', str(e))
                tensor_a_size = int(match.group(1))
                tensor_b_size = int(match.group(2))
                dimension = int(match.group(3))
                match dimension:
                    case 2 | 1:
                        padd = True
                        if trial > 1: 
                            if verbose > 0: print_flush(f"------------------- Trial {trial}  -----------------")
                            if tensor_a_size > tensor_a_size_old:
                                if verbose > 0: print_flush(f"------------------- Trial {trial} | a > a_old -----------------")
                                padd = False
                                y = y [ ..., : tensor_a_size - tensor_b_size]
                                if verbose > 0: print_flush(f"------------------- Trial {trial} |a > a_old | Reduced |  y ~ {y.shape} -----------------")
                        if padd:
                            if verbose > 0: print_flush(f"------------------- Trial {trial} | Padd -----------------")
                            if tensor_a_size > tensor_b_size: 
                                if verbose > 0: print_flush(f"------------------- Trial {trial} | Padd | a > b -----------------")
                                padd_step = tensor_a_size - tensor_b_size
                            y = torch.nn.functional.pad(y,(0,padd_step))
                        tensor_a_size_old = tensor_a_size
                    #case 1: 
                    #    if verbose > 0:
                    #        print_flush(f"sure_eval_moment | Trial {trial} | Error dimension 0 | mask ~ {mask.shape} | mask_input ~ {input_mask.shape} | batch ~ {y.shape}")
                    #        if mask.shape[1] < y.shape[2]: mask = torch.nn.functional.pad(mask,(0,y.shape[2]-mask.shape[1]))
                    #        if input_mask.shape[2] < y.shape[2]: mask = torch.nn.functional.pad(input_mask,(0,y.shape[2]-input_mask.shape[2]))

                    case 0:
                        if verbose > 0: 
                            print_flush(f"sure_eval_moment | Trial {trial} | Error dimension 0 | mask ~ {mask.shape} | mask_input ~ {input_mask.shape} | batch ~ {y.shape}")                    
                        if mask.shape[0] > y.shape[0]:
                            mask = mask[:y.shape[0]]
                        if input_mask.shape[0] > y.shape[0]:
                            input_mask = input_mask[:y.shape[0]]
                        
                        if mask.shape[0] < y.shape[0]:
                            extra_rows_shape = (-mask.shape[0]+y.shape[0],mask.shape[1])
                            if verbose > 0: print_flush(f"sure_eval_moment | Trial {trial} | Mask lower than batch | rows to add: {extra_rows_shape }")                    
                            extra_rows = torch.zeros(extra_rows_shape, dtype = torch.float32)
                            mask = torch.cat((mask, extra_rows), dim=0)
                        if input_mask.shape[0] < y.shape[0]:
                            extra_rows_shape = (-input_mask.shape[0]+y.shape[0],y.shape[1], y.shape[2])
                            if verbose > 0: print_flush(f"sure_eval_moment | Trial {trial} | Mask lower than batch | rows to add: {extra_rows_shape }")                    
                            extra_rows = torch.zeros(extra_rows_shape, dtype = torch.float32)
                            input_mask = torch.cat((input_mask, extra_rows), dim=0)
            else:
                if verbose > 0: 
                    print_flush("Not the usual error. No padding, just fail")
                raise
        if verbose > 0: print_flush(f"sure_eval_moment -->")
    y = y_copy
    if not cpu: y.to("cuda")
    return output

### Getting the embeddings (activations) from the encoder

In [57]:
#| export
from fastai.learner import Learner
from tsai.data.core import TSDataLoaders

In [58]:
#| export
def get_enc_embs_ensure_batch_size_(
    dls        : TSDataLoaders,
    batch_size : int = None,
    verbose    : int = 0
) -> None:
    if batch_size is None:
        if verbose > 1: 
            print_flush("[ Get Encoder Embeddings Ensure Batch Size ] No batch size proposed")
        if dls.bs == 0: 
            if verbose > 1: 
                print_flush("[ Get Encoder Embeddings Ensure Batch Size ] Using value 64 as 0 is not a valid value.")
            enc_learn.dls.bs = 64
        elif verbose > 1: 
            print_flush(f"[ Get Encoder Embeddings Ensure Batch Size ] Using the original value: {dls.bs}")
    else:
        dls.bs = batch_size
        if verbose > 1: 
            print_flush(f"[ Get Encoder Embeddings Ensure Batch Size ] Batch size proposed. Using {dls.bs}")

In [59]:
#| export
def get_enc_embs_MVP(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    module          : str  = None, 
    cpu             : bool = False, 
    average_seq_dim : bool = True, 
    to_numpy        : bool = True,
    batch_size      : int  = None,
    verbose         : int  = 0
):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
        - `batch_size`: force data loader to use the input batch size
        - `verbose`: print flag. More big, more information.
    """
    
    if cpu:
        if verbose > 0: print_flush("[ Get Encoder Embeddings ] CPU")
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        if verbose > 0: print_flush("[ Get Encoder Embeddings ] --> GPU")
        if verbose > 1: print_flush("[ Get Encoder Embeddings ] GPU | Ensure empty cache")
        torch.cuda.empty_cache()
        if verbose > 1: print_flush("[ Get Encoder Embeddings ] GPU | Move & exec into CUDA")
        enc_learn.dls.cuda()
        enc_learn.cuda()
        if torch.cuda.is_available():
            if verbose > 1: 
                print_flush("[ Get Encoder Embeddings ] GPU | CUDA is available")
                print_flush(f"[ Get Encoder Embeddings ] GPU | CUDA is available | current device id {torch.cuda.current_device()}")
                print_flush(f"[ Get Encoder Embeddings ] GPU | CUDA is available | current device name {torch.cuda.get_device_name(torch.cuda.current_device())}")            
        else:
            if verbose > 1: print_flush("[ Get Encoder Embeddings ] GPU | CUDA is not available")
        if verbose > 0: print_flush("[ Get Encoder Embeddings ] GPU -->")

    #if verbose > 0: print_flush("[ Get Encoder Embeddings ] Ensure the correct batch size")
    #get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)
    
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] Set dataloader from X (enc_learn does not contain dls)")
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    get_enc_embs_ensure_batch_size_(aux_dl, batch_size, verbose)
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] Get module")
    module = nested_attr(enc_learn.model,ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) if module is None else module
    
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads ")
    if verbose > 1: print_flush(f"[ Get Encoder Embeddings ] get_acts_and_grads bs = {aux_dl.bs}")
    
    embs = [
        get_acts_and_grads(
            model   = enc_learn.model,
            modules = module,
            x       = xb[0], 
            cpu     = cpu
        )[0] 
        for xb in aux_dl
    ]
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | --> Concat")
    if not cpu:
        if verbose > 1: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory")
        total_emb_size = sum([emb.element_size() * emb.nelement() for emb in embs])
        free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()
        if (total_emb_size < free_memory):
            if verbose > 1: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory | Fits in GPU -> Computing in GPU")
            embs=[emb.cuda() for emb in embs]
        else:
            if verbose > 1: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | Check neccesary & free memory | Does not fit in GPU -> Computing in CPU")
            embs=[emb.cpu() for emb in embs]
    if verbose > 1: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat | to_concat")
    embs = to_concat(embs)
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] get_acts_and_grads | Concat -->")
    
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] Reduce to 2 dimensions.")
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    if verbose > 0: print_flush("[ Get Encoder Embeddings ] Ensure CPU saving & numpy format")
    if to_numpy: embs = embs.numpy() if cpu else embs.cpu().numpy()
    return embs

In [60]:
#| export

def get_enc_embs_MVP_set_stride_set_batch_size(
    X                  : List [ List [ List [ float ] ] ], 
    enc_learn          : Learner, 
    stride             : int, 
    batch_size         : int, 
    module             : str  = None, 
    cpu                : bool = False, 
    average_seq_dim    : bool = True, 
    to_numpy           : bool = True, 
    verbose            : int  = 0, 
    time_flag          : bool = False, 
    chunk_size         : int  = 0, 
    check_memory_usage : bool = False
):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `X`: encoder input
        - `enc_learn`: trained encoder
        - `stride`: stride used for the training. Neccesary for adjusting the encoder input
        - `batch_size`: value to force the dataloader to use.
        - `module`: for geting the embeddings of an specific layer.
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
        - `verbose`: For printing messages. More big, more messages.
        - `time_flag`: To take note of the execution time required by this function
        - `chunk_size`: For spliting the embedings reading in batches of `chunk_size` size.
        - `check_memory_usage`: For showing messages of the current state of the memory.
    """
    if time_flag:
        t_start = time.time()
    if verbose > 0:
        print_flush("--> get_enc_embs_set_stride_set_batch_size")
    if check_memory_usage: gpu_memory_status()
    X = X[::stride]
    enc_learn.dls.bs = batch_size 

    get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)
    
    if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Check CUDA | X ~ ", X.shape[0])
    if cpu:
        if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Get enc embs CPU")
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        if torch.cuda.is_available():
            if verbose > 0: 
                print_flush("get_enc_embs_set_stride_set_batch_size | CUDA device id:", torch.cuda.current_device())
                print_flush("get_enc_embs_set_stride_set_batch_size | CUDA device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))
                print_flush("get_enc_embs_set_stride_set_batch_size | Ensure empty cache & move 2 GPU")
            torch.cuda.empty_cache()
            enc_learn.dls.cuda()
            enc_learn.cuda()
        else:
            if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | No cuda available. Set CPU = true")
            cpu = True
            
    get_enc_embs_ensure_batch_size_(enc_learn.dls, batch_size, verbose)

    if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Set dataset from X (enc_learn does not contain dls)")
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    aux_dl.bs = enc_learn.dls.bs if enc_learn.dls.bs>0 else 64
    if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Get module")
    module = nested_attr(enc_learn.model,ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) if module is None else module
    
    if verbose > 0: 
        #print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | module ", module)
        print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl len", len(aux_dl))
        print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl.batch_len ", len(next(iter(aux_dl))))
        print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl.bs ", aux_dl.bs)
        if (not cpu):
            total = torch.cuda.get_device_properties(device).total_memory
            used = torch.cuda.memory_allocated(torch.cuda.current_device())
            reserved = torch.cuda.memory_reserved(torch.cuda.current_device())
            print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | total_mem ", total)
            print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | used_mem ", used)
            print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | reserved_mem ", reserved)
            print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | available_mem ", total-reserved)
            sys.stdout.flush()
                                              
    if (cpu or ( chunk_size == 0 )):
        embs = [
            get_acts_and_grads(
                model=enc_learn.model,
                modules=module, 
                x=xb[0], 
                cpu=cpu
            )[0] 
            for xb in aux_dl
        ]
        if not cpu: embs=[emb.cpu() for emb in embs]
    else:
        embs = []
        total_chunks=max(1,round(len(X)/chunk_size))
        if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | aux_dl len | " + str(len(X)) + " chunk size: " + str(chunk_size) + " => " + str(total_chunks) + " chunks")
        for i in range(0, total_chunks):
            if verbose > 0: 
                print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | Chunk [ " + str(i) + "/"+str(total_chunks)+"] => " + str(round(i*100/total_chunks)) + "%")
                sys.stdout.flush()
            chunk = [batch for (n, batch) in enumerate(aux_dl) if (chunk_size*i <= n  and chunk_size*(i+1) > n) ]
            chunk_embs = [
                get_acts_and_grads(
                    model=enc_learn.model,
                    modules=module,
                    x=xb[0], 
                    cpu=cpu
                )[0]
                for xb in chunk
            ]
            # Mueve los embeddings del bloque a la CPU
            chunk_embs = [emb.cpu() for emb in chunk_embs]
            embs.extend(chunk_embs)
            torch.cuda.empty_cache()
        if verbose > 0: 
            print_flush("get_enc_embs_set_stride_set_batch_size | Get acts and grads | 100%")
            sys.stdout.flush()
    
    if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | concat embeddings")
    
    embs = to_concat(embs)
    
    if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Reduce")
    
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    
    if verbose > 0: print_flush("get_enc_embs_set_stride_set_batch_size | Convert to numpy")
    
    if to_numpy: 
        if cpu or chunk_size > 0:
            embs = embs.numpy() 
        else: 
            embs = embs.cpu().numpy()
            torch.cuda.empty_cache()
    if time_flag:
        t = time.time()-t_start
        if verbose > 0:
            print_flush("get_enc_embs_set_stride_set_batch_size " + str(t) + " seconds -->")
        else:
            print_flush("get_enc_embs_set_stride_set_batch_size " + str(t) + " seconds")
    if check_memory_usage: gpu_memory_status()
    if verbose > 0: 
        print_flush("get_enc_embs_set_stride_set_batch_size -->")
    return embs

In [61]:
#| export
def get_enc_embs_moment(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    cpu             : bool = False, 
    to_numpy        : bool = True,
    verbose         : int  = 0,
    average_seq_dim : bool = True
):
    if verbose > 0: 
        print_flush("--> get_enc_embs_moment")
    # Move tensor and model to GPU
    if cpu or not torch.cuda.is_available():
        if verbose > 0: 
            print_flush("get_enc_embs_moment | Using CPU (maybe no cuda available)")
        cpu = True
        enc_learn.cpu()
    else:
        if verbose > 0: 
            print_flush("get_enc_embs_moment | Using CUDA")
        enc_learn.to("cuda")
    if verbose > 0: print_flush("get_enc_embs_moment | Convert y")
    enc_learn.eval()
    if cpu:
        y = torch.from_numpy(X).cpu().float()
    else:
        y = torch.from_numpy(X).to("cuda").float()
    # Get output
    with torch.no_grad():
        if verbose > 0: 
            print_flush("get_enc_embs_moment | Get outputs")
        outputs = enc_learn(y)
        if verbose > 0:
            print_flush(f"get_enc_embs_moment | Final shape: X ~ {y.shape}")
                
    #| move tensors and models back to CPU
    if not cpu:
        y = y.detach().cpu().numpy()
    if verbose > 0: 
        print_flush("get_enc_embs_moment | Get Embeddings")
    embeddings = outputs.embeddings.detach().cpu()
    if average_seq_dim: 
        embeddings = embeddings.mean(dim = 1)
    if to_numpy:
        embeddings = embeddings.cpu().numpy()
    if verbose > 0: 
        print_flush("get_enc_embs_moment -->")
    return embeddings

In [62]:
#| export
def get_enc_embs_moment_reconstruction(
    X               : List [ List [ List [ float ] ] ], 
    enc_learn       : Learner, 
    cpu             : bool = False, 
    to_numpy        : bool = True,
    verbose         : int  = 0,
    average_seq_dim : bool = True,
    padd_step       : int  = 2
):
    """
    For reconstruction sometimes mask get invalid values
    To avoid them, the last dimension (sequence length) is padded with 0's until the error is skippedd
    It should only get one iteration as it seems to be some MOMENT internal configuration for patches.
    """
    if cpu:
        enc_learn.cpu()
        y = torch.from_numpy(X).cpu().float()
    else:
        enc_learn.to("cuda")
        y = torch.from_numpy(X).to("cuda").float()
    embs = get_acts_moment(
        enc_learn = enc_learn, 
        cpu = cpu, 
        verbose = verbose, 
        y = y, 
        mask = None,
        padd_step = padd_step,
        retry = False ,
        max_trials = 5
    )
    if average_seq_dim: 
        embs = embs.mean(dim = 1).mean(dim = 1)
    if to_numpy:
        embs = embs.cpu().numpy()
    return embs

---> TODO: averiguar de qué module salen realmente los embeddings y usar el get_acts_and_grads como en MVP <---

In [63]:
#| export
import uni2ts.model.moirai.module as moirai
import uni2ts.model.moirai.forecast as moirai_forecast

In [64]:
#| export
import torch.profiler as profiler

In [65]:
#| export
def watch_gpu(func, **kwargs):
    """
    Wrapper to execute GPU profiler
    Parameters: 
    - func: function to monitor
    - kwargs: func parameters
    Returns:
    - result of /func/.
    """
    with profiler.profile(
        activities=[profiler.ProfilerActivity.CPU, profiler.ProfilerActivity.CUDA],
        schedule=profiler.schedule(wait=1, warmup=1, active=3, repeat=2),  # Configuración de ciclos
        on_trace_ready=profiler.tensorboard_trace_handler('./log_dir'),  # Guarda los resultados en un archivo para visualización
        record_shapes=True,  # Registra la forma de los tensores
        profile_memory=True,  # Perfil de memoria
        with_stack=True  # Incluye la información de la pila
    ) as prof:
        # Ejecuta la función dentro del perfilador
        result = func(**kwargs)
    
    # Mostrar el uso de la GPU durante y después de la ejecución
    print_flush(prof.key_averages().table(sort_by="cuda_memory_usage", row_limit=10))
    return result

In [66]:
#| export
def get_enc_embs_moirai(
    enc_input       : List [ List [ List [ Float ] ] ], 
    enc_model       : moirai.MoiraiModule, 
    cpu             : False,
    average_seq_dim : bool = True, 
    verbose         : int  = 0,
    to_numpy        : bool = True,
    patch_size      : int  = 8,
    time            : bool = False
):
    if time: 
        timer = Time()
        timer.start()
    if verbose > 0: 
        print_flush("--> get_enc_embs_moirai")
    # Move tensor and model to GPU
    past_target = einops.rearrange(
        torch.as_tensor(enc_input, dtype = torch.float32),
        "n_windows n_vars window_size -> n_windows window_size n_vars"
    )
    if cpu or not torch.cuda.is_available():
        if verbose > 0: print_flush("get_enc_embs_moirai | Using CPU (maybe no cuda available)")
        cpu = True
        enc_model.cpu()
        past_target.cpu()
    else:
        if verbose > 0: print_flush("get_enc_embs_moirai | Using CUDA")
        enc_model.to("cuda")
        past_target.to("cuda")
        
    if verbose > 0: print_flush("get_enc_embs_moirai | Get Outputs")

    
    past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
    past_is_pad = torch.zeros_like(past_target, dtype=torch.bool)[...,:,-1] # Kill last dimension

    if (verbose > 1):
        print_flush(f"--> get_enc_embs_moirai | past_target ~ {past_target.shape}")
        print_flush(f"--> get_enc_embs_moirai | past_observed_target ~ {past_observed_target.shape}")
        print_flush(f"--> get_enc_embs_moirai | past_is_pad ~ {past_is_pad.shape}")
        print_flush(f"--> get_enc_embs_moirai | Auxiliar model")
        print_flush(f"--> get_enc_embs_moirai | Auxiliar model | Before Memory:")
        gpu_memory_status()
    
    # Auxiliar model for conversions just to ensure correct sizes
    #not neccesary, is the same module initially downloaded...
    #module = moirai.MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-small")
    
    forecast_model =  moirai_forecast.MoiraiForecast(
        module=enc_model,
        prediction_length=past_target.shape[2], #random, just for getting the model
        context_length=past_target.shape[1],
        patch_size=patch_size,
        num_samples=100, #Random, is the number of forecasting, not interesting for us
        target_dim=past_target.shape[2],
        feat_dynamic_real_dim=0,
        past_feat_dynamic_real_dim=0,
    )
    
    if verbose > 0:
        print_flush(f"--> get_enc_embs_moirai | Auxiliar model | After Memory:")
        gpu_memory_status()
        print_flush(f"--> get_enc_embs_moirai | Convert sizes")
    (
    target,
    observed_mask,
    sample_id,
    time_id,
    variate_id,
    prediction_mask,
    ) = forecast_model._convert(
        patch_size,
        past_target,
        past_observed_target,
        past_is_pad
    )
    if verbose > 1:
        print_flush(f"get_enc_embs_moirai | target ~ {target.shape}")
        print_flush(f"get_enc_embs_moirai | observed_mask ~ {observed_mask.shape}")
        print_flush(f"get_enc_embs_moirai | sample_id ~ {sample_id.shape}")
        print_flush(f"get_enc_embs_moirai | time_id ~ {time_id.shape}")
        print_flush(f"get_enc_embs_moirai | variate_id ~ {variate_id.shape}")
        print_flush(f"get_enc_embs_moirai | prediction_mask ~ {prediction_mask.shape}")
        gpu_memory_status()
    forecast_model = None
    torch.cuda.empty_cache()
    if verbose > 0:
        print_flush(f"--> get_enc_embs_moirai | Delete Auxiliar model | After Memory:")
        gpu_memory_status()
    
    model_kwargs={
        'target': target, 
        'observed_mask': observed_mask,
        'sample_id': sample_id,
        'time_id': time_id,
        'variate_id': variate_id,
        'prediction_mask': prediction_mask,
        'patch_size': torch.ones_like(sample_id, dtype = torch.float32)*patch_size
    } 
    if verbose > 0: 
        print_flush(f"get_enc_embs_moirai | About to get activations")
    acts = get_acts(
        model  = enc_model, 
        module = enc_model.encoder.norm, 
        cpu    = cpu,
        verbose = verbose,
        retry = True,
        acts_indices = [0],
        **model_kwargs #Parameters of the model
    )
    
    embs = acts
    acts = None
    if average_seq_dim :
        if verbose > 0: 
            print_flush(f"get_enc_embs_moirai | About to reduce activations")
        embs = embs.mean(dim = 1)
    
    if not cpu:
        #print_flush(f"get_enc_embs_moirai | enc_input to cpu")
        #enc_input.cpu()
        print_flush(f"get_enc_embs_moirai | enc_model to cpu")
        enc_model.cpu()
        print_flush(f"get_enc_embs_moirai | torch cuda empty cache")
        torch.cuda.empty_cache()
    if to_numpy: 
        if cpu > 0:
            embs = embs.numpy() 
        else: 
            embs = embs.cpu().numpy()
            torch.cuda.empty_cache()
    if verbose > 0: 
        print_flush(f"get_enc_embs_moirai | embs ~ {embs.shape}")
        print_flush("get_enc_embs_moirai -->")
    return embs

In [67]:
#| export 
def get_enc_embs(
    X               , 
    enc_learn       : Learner, 
    module          : str  = None, 
    cpu             : bool = False, 
    average_seq_dim : bool = True, 
    to_numpy        : bool = True,
    verbose         : int  = 0,
    **kwargs        
):
    embs = None
    enc_learn_class = str(enc_learn.__class__)[8:-2]
    match enc_learn_class:
        case "momentfm.models.moment.MOMENTPipeline":
            match enc_learn.task_name:
                case "embedding":
                    embs = get_enc_embs_moment(X, enc_learn, cpu, to_numpy, verbose, average_seq_dim, **kwargs)
                case "reconstruction":
                    embs = get_enc_embs_moment_reconstruction(X, enc_learn, cpu, to_numpy, verbose, average_seq_dim, **kwargs)
                case _:
                    print_flush(f"Model embeddings for moment-{enc_learn.task_name} is not yet implemented.")
        case "fastai.learner.Learner":
            embs = get_enc_embs_MVP_set_stride_set_batch_size(X, enc_learn, stride, batch_size, module, cpu, average_seq_dim, to_numpy, verbose, False, 0, False)
        case "uni2ts.model.moirai.module.MoiraiModule":
            embs = get_enc_embs_moirai(
                enc_input  = X, 
                enc_model  = enc_learn,
                cpu        = cpu, 
                average_seq_dim = average_seq_dim,
                verbose    = verbose,
                **kwargs
            )
        case _:
            print_flush(f"Model embeddings implementation is not yet implemented for {enc_learn_class}.")
    return embs

In [68]:
#| export
def get_enc_embs_set_stride_set_batch_size(
    X                  : List [ List [ List [ float ] ] ], 
    enc_learn          : Learner, 
    stride             : int, 
    batch_size         : int, 
    module             : str  = None, 
    cpu                : bool = False, 
    average_seq_dim    : bool = True, 
    to_numpy           : bool = True, 
    verbose            : int  = 0, 
    time_flag          : bool = False, 
    chunk_size         : int  = 0, 
    check_memory_usage : bool = False,
    **kwargs
):
    print_flush("--> get_enc_embs_set_stride_set_batch_size")
    embs = None
    enc_learn_class = str(enc_learn.__class__)[8:-2]
    match enc_learn_class:
        case "momentfm.models.moment.MOMENTPipeline":
            if verbose > 0: 
                print_flush(f"get_enc_embs_set_stride_set_batch_size | Moment | {average_seq_dim}")
            match enc_learn.task_name:
                case "embedding":
                    embs = get_enc_embs_moment( X = X, enc_learn = enc_learn, cpu = cpu, to_numpy = to_numpy, verbose = verbose, average_seq_dim = average_seq_dim)
                case "reconstruction":
                    embs = get_enc_embs_moment_reconstruction(X= X, enc_learn = enc_learn, cpu = cpu, to_numpy = to_numpy, verbose = verbose, average_seq_dim = average_seq_dim, **kwargs)
                case _:
                    print_flush(f"Model embeddings for moment-{enc_learn.task_name} is not yet implemented.")
        case "fastai.learner.Learner":
            if verbose > 0: 
                print_flush(f"get_enc_embs_set_stride_set_batch_size | MVP | {average_seq_dim}")
            embs = get_enc_embs_MVP_set_stride_set_batch_size(
                X = X, 
                enc_learn = enc_learn, 
                stride = stride, 
                batch_size = batch_size, 
                module = module, 
                cpu = cpu, 
                average_seq_dim = average_seq_dim,
                to_numpy = to_numpy, 
                verbose = verbose, 
                time_flag = time_flag, 
                chunk_size = chunk_size, 
                check_memory_usage = check_memory_usage
            )
        case "uni2ts.model.moirai.module.MoiraiModule":
            if verbose > 0: 
                print_flush(f"get_enc_embs_set_stride_set_batch_size | Moirai | {average_seq_dim}")
            embs = get_enc_embs_moirai(
                enc_input  = X, 
                enc_model  = enc_learn,
                cpu        = cpu, 
                average_seq_dim = average_seq_dim,
                verbose    = verbose,
                to_numpy = to_numpy,
                **kwargs
            )
        case _:
            print_flush(f"[ get_enc_embs_set_stride_set_batch_size ] Model embeddings implementation is not yet implemented for {enc_learn_class}.")
    # Ñapa: TODO: Gestionar que no se queden en memoria los modelos porque ocupan el 40% de la GPU al llamarlos desde R
    if verbose > 0: print_flush(f"get_enc_embs_set_stride_set_batch_size | Before moving to CPU | embs~{embs.shape}")
    if cpu:
        #X.cpu()
        enc_learn.cpu()
        try: 
            enc_lear.dls.cpu()
        except Exception as e: 
            print_flush(f"get_enc_embs_set_stride_set_batch_size | Exception: {e}")
        #kwargs_to_cpu_(**kwargs)
    if verbose > 0: print_flush(f"get_enc_embs_set_stride_set_batch_size | embs~{embs.shape} -->")
    return embs

## Fine-tunning
> Take a look on [HuggingFace - Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training) if not used to few-shot learning or fine-tuning models.

Steps: 

1) Prepare the dataset
2) Batch the data
   - Remember splitting between train & test dataset
   - Remember to use DataLoader to iterate over batches
4) Load the trained model and check if any modification is needed
   - Check wether any layer may be substituted by an "identity" if not needed for your case
   - Check if any dimension in a conversion layer may be changed to fit your dataset.
5) Select an optimizer from torch.optim (Adam)
6) ¿If using transformer, lr_scheduler? 
7) Training loop
     

### Utils

In [69]:
#| export
def random_windows(X : List [ List [ List [ float ]]], n_windows = None, percent = 0.2, verbose = 0):
    """
    Parameters: 
    - X: Numpy array of windows. Expected shape: [batch_size or n_samples, n_vars, window_len]
    Given a numpy array of windows, selects:
    - n_windows random windows from the array, if n_windows is given.
    - ceil(percent*len(X)) random windows otherwise
    """
    n_windows = int(min(X.shape[0], n_windows) if n_windows is not None else np.ceil(percent*X.shape[0]))
    if verbose > 0: print_flush(f"Random windows | n_windows: {n_windows}")
    random_indices = np.random.randint(0, int(X.shape[0]), n_windows)
    windows = X[ random_indices ]
    windows = torch.from_numpy(windows)
    if verbose > 0: print_flush(f"windows ~ {windows.shape}")
    return windows

### Moment
> Follow the tutorial in the original repository: [Moment - Imputation](https://github.com/moment-timeseries-foundation-model/moment/blob/main/tutorials/imputation.ipynb).

In [70]:
# Todo: poner en docker
#! pip install evaluate

In [71]:
#| export
from momentfm.utils.masking import Masking
from tqdm.auto import tqdm
from transformers import get_scheduler
import evaluate

In [72]:
#| export
def fine_tune_moment_compute_loss_check_sizes_(batch, output, verbose = 0):
    if verbose > 0: print_flush("--> fine_tune_moment_compute_loss_check_sizes_")
    b = batch.clone()
    b_2 = batch.shape[2]
    re_2 = output.reconstruction.shape[2]
    if b_2 > re_2:
        if verbose > 0: print_flush(f" Fine tune loop | TODO: Why? Original {b_2} > {re_2}  Reconstruction")
        b = b[...,:re_2]
    elif re_2 > b_2:
        if verbose > 1: print_flush(f" Fine tune loop | Why ? Original {b_2} < {re_2} Reconstruction ? Padding")
        output.reconstruction = output.reconstruction[...,:b_2]
    else: 
        if verbose > 1: print_flush(f" Fine tune loop | re_2 {re_2} == {b_2} y_2")
    if verbose > 1: 
        print_flush(f"---------- Checking loss  ------- | reconstruction ~ {output.reconstruction.shape} | original_ ~ {b.shape}")
    if verbose > 0: print_flush("fine_tune_moment_compute_loss_check_sizes_ -->")
    return b

In [73]:
#| export
def fine_tune_moment_compute_loss(batch, output, criterion = torch.nn.MSELoss, verbose = 0, input_mask = None, mask = None):
    if verbose > 0: print_flush("--> fine_tune_moment_compute_loss")
    b = fine_tune_moment_compute_loss_check_sizes_(batch = batch, output = output, verbose = verbose)
    if verbose > 0: print_flush(f"fine_tune_moment_compute_loss | b~{b.shape} | o~{output.reconstruction.shape}")
    compute_loss = criterion()
    recon_loss = compute_loss(output.reconstruction, b)
    batch_masks = output.input_mask if input_mask is None else input_mask
    mask = output.pretrain_mask if mask is None else mask
    if verbose > 1: print_flush(f"fine_tune_moment_compute_loss | batch ~ {b.shape}")
    if verbose > 1: print_flush(f"fine_tune_moment_compute_loss | batch_masks ~ {batch_masks.shape}")
    if verbose > 1: print_flush(f"fine_tune_moment_compute_loss | mask ~ {mask.shape}")
    observed_mask = batch_masks * (1-mask)
    masked_loss = observed_mask * recon_loss
    loss = masked_loss.nansum() / (observed_mask.nansum() + 1e-7)
    if verbose > 1: print_flush(f"fine_tune_moment_compute_loss | loss: {loss.item()}")
    if verbose > 0: print_flush(f"Loss type: {type(loss)}")  # Debe ser <class 'torch.Tensor'>
    if verbose > 0: print_flush("fine_tune_moment_compute_loss -->")
    return loss

In [ ]:
#| export
def fine_tune_moment_eval_preprocess(
    predictions : List [ List [ float ]],
    references : List [ List [ float ]],
    verbose : int = 0
):
    """
    Parameters:
    - predictions torch (float)
    - references torch (float)
    Returns: 
        - Predictions and references ensuring same shape and no NaN values. 
        - Uses the shape of the smallest torch for the modification.
    """
    if verbose > 0: 
        print(f"fine_tune_moment_eval | Before reshape | preds~{predictions.shape}")            
        print(f"fine_tune_moment_eval | Before reshape | refs~{references.shape}")
    predictions = einops.rearrange(predictions, "b v w -> (b v) w")
    references = einops.rearrange(references, "b v w -> (b v) w")
    # Avoid NaN 
    if predictions.shape[1] > references.shape[1]: predictions = predictions[:,:references.shape[1]]
    if predictions.shape[1] < references.shape[1]: references = references[:,:predictions.shape[1]]
    if verbose > 0: 
        print(f"Eval | After reshape | preds~{predictions.shape}")
        print(f"Eval | After reshape | refs~{references.shape}")
        
    nan_mask = torch.isnan(predictions) | torch.isnan(references)
    predictions = torch.where(nan_mask, torch.tensor(0.0), predictions)
    references = torch.where(nan_mask, torch.tensor(0.0), references)
    if verbose > 0: 
        print(f"Eval | After NaN | preds~{predictions.shape}")
        print(f"Eval | After NaN | refs~{references.shape}")
    return predictions, references

In [74]:
#| export
def fine_tune_moment_eval_(
    enc_learn : Learner,
    dl_eval   : DataLoader,
    num_epochs: int = 1,
    cpu       : bool = False,
    verbose   : int = 0
):
    # Select device
    device = "cpu" if cpu else torch.cuda.current_device()
    # Load metrics
    mse_metric = evaluate.load('mse', "multilist")
    rmse_metric = evaluate.load('mse', "multilist")
    mae_metric = evaluate.load('mae', "multilist")
    smape_metric = evaluate.load("smape", "multilist")
    num_evaluation_steps = len(dl_eval)
    progress_bar = tqdm(range(num_evaluation_steps))
    # Predict evaluation dataset
    enc_learn.eval()
    for batch in dl_eval:
        with torch.no_grad():
            output = sure_eval_moment(
                enc_learn = enc_learn, 
                cpu = cpu,
                verbose = verbose,                     
                y = batch, 
                input_mask = None,
                mask = None,
                padd_step = 100, 
                max_trials = 5, 
                acts_indices = None
            )
        logits = output.logits
        predictions = output.reconstruction
        references = batch
        predictions, references = fine_tune_moment_eval_preprocess(predictions = predictions, references = references, verbose = verbose)
        mse_metric.add_batch(predictions=predictions, references = references)
        rmse_metric.add_batch(predictions=predictions, references = references)
        mae_metric.add_batch(predictions=predictions, references = references)
        smape_metric.add_batch(predictions=predictions, references = references)
        progress_bar.update(1)
    mse   = mse_metric.compute(squared = False)
    rmse  = rmse_metric.compute(squared = True)
    mae   = mae_metric.compute()
    smape = smape_metric.compute()
    eval_results = {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "smape": smape
    }
    enc_learn.train()
    return eval_results

In [75]:
#| export
def fine_tune_moment_train_(
    enc_learn : Learner, 
    dl_train: DataLoader,
    ds_train,
    batch_size : int,
    num_epochs : int = 1,
    criterion = torch.nn.MSELoss, 
    optimizer = torch.optim.Adam, 
    lr = 1e-4, 
    lr_scheduler_flag = False, 
    lr_scheduler_name = "linear",
    lr_scheduler_num_warmup_steps = 0,
    cpu = False,
    verbose = 0
):
    # Optimizer and learning rate scheduler
    optimizer = torch.optim.Adam(enc_learn.parameters(), lr)
    num_training_steps = num_epochs * len(dl_train)
    losses = []
    if lr_scheduler_flag:
        lr_scheduler = get_scheduler(
            name = lr_scheduler_name,
            optimizer = optimizer,
            num_warmup_steps = lr_scheduler_num_warmup_steps,
            num_training_steps = num_training_steps
        )
    # Select device
    device = "cpu" if cpu else torch.cuda.current_device()
        
    # Training loop
    if verbose > 1: print_flush("fine_tune_moment_ | Training loop")
    progress_bar = tqdm(range(num_training_steps))
    # Masks
    mask_generator = Masking(mask_ratio = 0.3)
    n_samples, n_channels, window_size = ds_train.shape
    batch_masks = torch.ones(
        (batch_size, window_size), 
        device = device
    ).long()
        
    if verbose > 1: print_flush("fine_tune_moment_ | Fine tune loop")        
    for epoch in range(num_epochs):
        for batch in dl_train:
            batch.to(device)
            bms = batch_masks
            if batch.shape[0] < batch_masks.shape[0]:  
                bms = batch_masks[:batch.shape[0]]
            if verbose > 1: 
                print_flush(
                    f"fine_tune_moment_ | Fine tune loop | batch ~ {batch.shape} | batch_masks ~ {bms.shape}"
                )
            mask = mask_generator.generate_mask(
                x = batch,
                input_mask = bms
            )
            if mask.shape[0] < bms.shape[0]:  bms = batch_masks[:mask.shape[0]]
            if mask.shape[1]  < batch_masks.shape[1] : mask = torch.nn.functional.pad(mask,(0,batch_masks.shape[1]-mask.shape[1]))
            if verbose > 1: 
                print_flush(
                    f"fine_tune_moment_ | Fine tune loop | batch ~ {batch.shape} | batch_masks ~ {bms.shape} | mask ~ {mask.shape}"
                )
            output = sure_eval_moment(
                enc_learn = enc_learn, 
                cpu = cpu,
                verbose = verbose-1, 
                y = batch, 
                input_mask = batch_masks, # None
                mask = mask, # None
                padd_step = 100, 
                max_trials = 5, 
                acts_indices = None
            )
            # Compute output loss
            loss = fine_tune_moment_compute_loss(batch, output, criterion, verbose = verbose, input_mask = bms, mask = mask )
            losses.append(loss.item())
            # Update weights
            optimizer.zero_grad()
            loss.backward()
            # Optimize
            optimizer.step()
            if lr_scheduler_flag: lr_scheduler.step()
            progress_bar.update(1)
    return losses

In [ ]:
#| export
from torch.nn.modules.loss import _Loss

In [76]:
#| export
def fine_tune_moment_(
    X                               : List [ List [ List [ float ]]], 
    enc_learn                       : Learner, 
    stride                          : int   = 1,      
    batch_size                      : int   = 32,
    cpu                             : bool  = False,
    to_numpy                        : bool  = True, 
    verbose                         : int   = 0, 
    time_flag                       : bool  = False,
    n_windows                       : int   = None,
    n_windows_percent               : float = 0.2,
    validation_percent              : float = 0.2, 
    training_percent                : float = 0.2,
    num_epochs                      : int   = 3,
    shot                            : bool  = True,
    eval                            : bool  = True,
    criterion                       : _Loss = torch.nn.MSELoss, 
    optimizer                               = torch.optim.Adam, 
    lr                              : float =  1e-4, 
    lr_scheduler_flag               : bool  = False, 
    lr_scheduler_name               : str   = "linear",
    lr_scheduler_num_warmup_steps   : int   = 0,
):   
    if verbose > 0: print_flush("--> fine_tune_moment_")
    t_shot = 0
    t_eval_1 = 0
    t_eval_2 = 0
    losses = []
    eval_results_pre = ""
    eval_results_post = ""

    if time_flag:
        timer = Time()
    # Prepare the dataset
    train_split_index = min(X.shape[0], n_windows) if n_windows is not None else np.ceil(training_percent * X.shape[0])
    eval_split_index = min(X.shape[0], n_windows) if n_windows is not None else np.ceil(validation_percent * X.shape[0])
    train_split_index = int(train_split_index)
    eval_split_index = int(eval_split_index)
    if shot: ds_train = X[:train_split_index]
    if eval: ds_test  = torch.from_numpy(X[:eval_split_index]).float()
    # -- Select only the small percentage for few-shot
    if verbose > 1: print_flush("fine_tune_moment_ | Random windows")
    if shot:
        ds_train = random_windows(ds_train, n_windows, n_windows_percent, verbose-1)
        ds_train = ds_train.float()
        # Create the dataloader
        dl_train = DataLoader(ds_train, batch_size = batch_size, shuffle = True)
    if eval: dl_eval  = DataLoader(ds_test, batch_size = batch_size, shuffle = False)
    if eval:
        if time_flag: timer.start()
        eval_results_pre = fine_tune_moment_eval_(
            enc_learn = enc_learn,
            dl_eval = dl_eval,
            num_epochs = num_epochs,
            cpu = cpu,
            verbose = verbose-1
        )
        if time_flag: 
            timer.end()
            t_eval_1 = timer.duration()
            if verbose > 0: timer.show()
    if shot:
        if time_flag: timer.start()
        losses = fine_tune_moment_train_(
            enc_learn = enc_learn,
            dl_train = dl_train,
            ds_train = ds_train,
            batch_size = batch_size,
            num_epochs = num_epochs,
            criterion = torch.nn.MSELoss, 
            optimizer = torch.optim.Adam, 
            lr = 1e-4, 
            lr_scheduler_flag = False, 
            lr_scheduler_name = "linear",
            lr_scheduler_num_warmup_steps = 0,
            cpu = cpu,
            verbose = verbose-1
        )
        if time_flag:
            timer.end()
            t_shot = timer.duration()
            if verbose > 0: timer.show()
    if eval:    
        if time_flag: timer.start()
        eval_results_post = fine_tune_moment_eval_(
            enc_learn = enc_learn,
            dl_eval = dl_eval,
            num_epochs = num_epochs,
            cpu = cpu,
            verbose = verbose-1
        )
        if time_flag:
            timer.end()
            t_eval_2 = timer.duration()
            if verbose > 0: timer.show()
        if verbose > 0: print("Evaluation summary")
    return losses, eval_results_pre, eval_results_post, t_shot, t_eval_1, t_eval_2

In [77]:
#| hide
#import wandb
#from dvats.utils import *
#wandb_api = wandb.Api()
#enc_artifact = wandb_api.artifact('deepvats/mvp-SWV:latest')
#enc_learner = enc_artifact.to_obj()
#X = torch.rand(9, 1, 48)

In [78]:
#| hide
#%time
#embs = get_enc_embs(X, enc_learner, cpu=True)
#test_eq(embs.shape[0], X.shape[0])
#embs.shape, embs.__class__

In [79]:
#| hide
#%%time #TODO dont work with nb2py
#embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=False)
#test_eq(embs.shape[0], X.shape[0])
#embs.shape, embs.__class__, embs.device

In [80]:
#| hide 
#%%time #TODO --> dont works with nb2py
#embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=True)
#test_eq(embs.shape[0], X.shape[0])
#embs.shape, embs.__class__